<em><sub>This page is available as an executable or viewable <strong>Jupyter Notebook</strong>:</sub></em>
<br/><br/>
<a href="https://mybinder.org/v2/gh/Cottand/markov-lock-simulation/v1?filepath=queue.ipynb"
   target="_parent"> 
   <img align="left" 
        src="https://mybinder.org/badge_logo.svg">
</a>
<a href="https://nbviewer.jupyter.org/github/Cottand/markov-lock-simulation/blob/master/queue.ipynb" 
   target="_parent"> 
   <img align="right" 
        src="https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.png" 
        width="109" height="20">
</a>
<br/>
<br/>

In [72]:
%use lets-plot
%use kmath
%use fuel, krangl

import kotlin.ranges.IntRange

fun Collection<Double>.product() = reduce { acc, i ->  acc * i }

fun sum(range: IntRange, op: (Int) -> Double) = range.map(op).sum()

fun prod(range: IntRange, op: (Int) -> Double) = range.map(op).product()

operator fun IntRange.times(i: Int) = (0 until i).flatMap { this.toList() }

## `Queue` class

This class represents the context of our queue and its paramenters:

- $t_x$ (us): time, on average, that a core takes to run its non-critical section of the code (ie, job arrival time)
- $t_c$ (us): time, on average, that a core takes to run its _ciritical_ section of the code (which it can only do once it acquires the global lock)
- $t_u$ (us): time, on average, that it takes to update the cache of a core _after_ the global lock has been released. $ct_u/2$ is the time it takes for a core to grab the lock, when there are $c$ cores waiting for the lock
- $n$, or `queueSize`: the total number of cores in this model
- `constantRelease`: whether to model instead an 'ideal' model where the time it takes for a core to grab the lock is constant

Which in turns allows us to compute:

- $p_k$, or `p(k)`: the probability of the system being in state $k$ at any given time
- $\lambda_k$: the rate at which we jump from state $k$ to state $k+1$ (ie, the rate at which cores attempt to grab the lock)
- $r_k$: the rate at which we jump from state $k+1$ to state $k$ (ie, the rate at which cores release the lock)
- $W_Q$, or `avgWaitingTime`: the average waiting for the lock a core does
- $N$, or `avgBusyCores`: the average number of cores either using or waiting for the lock

In [73]:
class Queue(
    val tx: Double,
    val tc: Double,
    val tu: Double,
    val queueSize: Int,
    val constantRelease: Boolean = false
) {
    val n = queueSize
    
    val p0 by lazy {
        val sum = sum(1..queueSize) { k -> prod(0..k-1) { i -> lambda(i) / r(i + 1) } }
        1 / (1 + sum) 
    }
    
    
    fun lambda(k: Int) = (queueSize - k) / tx

    fun r(k: Int) = if (constantRelease) 1 / (tc + tu) else 2 / (2 * tc + (k - 1) * tu)

    fun p(k: Int) = if (k == 0) p0 else p0 * prod(0..k-1) { i -> lambda(i) / r(i + 1) }
    
    // N
    val avgBusyCores by lazy {
        sum(0..queueSize) { k -> k * p(k) }
    }
    
    // W_Q
    val avgWaitTime by lazy {
        val serviceTime = sum(0..queueSize) { k -> p(k) / r(k) }
        avgBusyCores / throughput - serviceTime
    }
    
    // W_U
    val avgUpdateTime by lazy {
        sum(1..queueSize) { k -> (k - 1) * tu * p(k) / 2.0 }
    }
    
    val utilisation by lazy { 1 - p0 }
    
    val throughput by lazy { sum(0..queueSize) { k -> lambda(k) * p(k) } }
    
    val speedup by lazy { n - avgBusyCores }
    
    companion object
}

fun Queue.Companion.with(tx: Int, n: Int, constantRelease: Boolean = false) = 
    Queue(tx.toDouble(), 2.0, 5.0, n, constantRelease)

### Our dataset

Three sets of queues with tx = 100, 300, 500 and n ranging from 1 to 32



In [74]:
val txs = listOf(100, 300, 500)
val range = 1..32

val (labels, queues) = txs.flatMap { tx ->
     range.map { n -> "tx=$tx" to Queue.with(tx, n) }
}.unzip()

val queuesConstant = txs.flatMap { tx ->
     range.map { n -> Queue.with(tx, n, constantRelease = true) }
}

// utility to select a queue's property
operator fun <T> Collection<Queue>.invoke(select: Queue.() -> T) = map { it.select() }

val data = mapOf(
    "n" to (range * 3),
    "label" to labels,
    "utilisation" to queues { utilisation },
    "utilisation (constant)" to queuesConstant { utilisation },
    "W_Q" to queues { avgWaitTime },
    "W_Q (constant)" to queuesConstant { avgWaitTime },
    "speedup" to queues { speedup },
    "speedup (constant)" to queuesConstant { speedup },
    "update" to queues { avgUpdateTime },
    "update (constant)" to queuesConstant { avgUpdateTime },
    "throughput" to queues { throughput },
    "throughput (constant)" to queuesConstant { throughput },
    "tu" to List(32 * 3) { 5.0 },
    "tu label" to List(32 * 3) { "tu" },
)

## Mean time spent waiting to acquire the lock, $W_Q$, in terms of the number of cores, $n$

In [75]:

val meanTimePlot = 
    lets_plot(data) { color = "label" } +
    geom_line { x = "n"; y = "W_Q" } +
    geom_line(linetype = "dashed") { x = "n"; y = "W_Q (constant)" } +
    xlab("Number of cores (n)") +
    ylab("Average waiting time (W_Q, in μs)") +
    scale_color_discrete(name = "tx") +
    ggsize(600, 300)
    
meanTimePlot


## Mean time spent updating the cores' cache, W_U, in terms of the number of cores, $n$

In [76]:

val updateTimePlot = 
    lets_plot(data) +
    geom_line { x = "n"; y = "update"; color = "label" } +
    geom_line(linetype = "dashed") { x = "n"; y = "update (constant)"; color = "label" } +
    geom_line(linetype = "dotted") { x = "n"; y = "tu"; color = "tu label" } +
    xlab("Number of cores (n)") +
    ylab("Average updating time (W_U, in μs)") +
    scale_color_discrete(name = "curves") +
    ggsize(600, 300)
    
updateTimePlot

## Speedup over a single core (given by $N - n$)  in terms of the number of cores, $n$

In [77]:
val speedupPlot =
    lets_plot(data) +
    geom_line { x = "n"; y = "speedup"; color = "label" } +
    geom_line(linetype = "dashed") { x = "n"; y = "speedup (constant)"; color = "label" } +
    xlab("Number of cores (n)") +
    ylab("Speedup over a single core") +
    scale_color_discrete(name = "tx") +
    ggsize(600, 300)
    
speedupPlot


## Throughput $\tau$, in terms of number of cores $n$ 

In [78]:
val throughputPlot =
    lets_plot(data) +
    geom_line { x = "n"; y = "throughput"; color = "label" } +
    geom_line(linetype = "dashed") { x = "n"; y = "throughput (constant)"; color = "label" } +
    xlab("Number of cores (n)") +
    ylab("Throughput (τ, in jobs/μs)") +
    scale_color_discrete(name = "tx") +
    ggsize(600, 300)
    
throughputPlot

In [79]:

ggsave(meanTimePlot, "meanTime1.png") 
ggsave(updateTimePlot, "updateTime.png") 
ggsave(speedupPlot, "speedup.png") 
ggsave(throughputPlot, "throughput.png")

/home/cottand/dev/sim-jupyter/lets-plot-images/throughput.png